In [2]:
print('hello')

hello


### How to Build a Advance Chatbot with Stored Memory using LangChain

This script demonstrates how to build a simple LLM-based chatbot with memory using LangChain. The chatbot:
- Can hold a conversation.
- Remembers previous interactions by storing memory.
- Saves memory into a JSON file for persistence.

A chatbot app is a type of software designed to simulate conversations with human users. It uses AI to interpret user input and provide relevant responses. These bots can be applied in customer service, information delivery, or task assistance. In essence, it’s like texting a smart assistant that can answer questions and help you out.

Concepts Covered:
- Chat Model vs LLM Model:
    - Chat models operate on structured messages.
    - LLM models operate on raw unstructured text.
    - Enables the chat model to retain and utilize previous interactions for context.


In [4]:
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


from dotenv import load_dotenv, find_dotenv

In [5]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [6]:
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [7]:
from langchain_groq import ChatGroq
chatbot = ChatGroq(
    model="llama-3.3-70b-versatile"
)

In [8]:
from langchain_core.messages import HumanMessage

messages_to_the_bot = [
    HumanMessage(content="My favorite color is green")
]

In [9]:
chatbot.invoke(messages_to_the_bot)

AIMessage(content="Green is a wonderful color, often associated with nature, growth, and harmony. It's a calming and balancing color that can evoke feelings of serenity and relaxation. What is it about the color green that you particularly like? Is it the vibrant tone of a freshly picked lime, the soothing hue of a forest, or something else entirely?", response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 40, 'total_tokens': 109, 'completion_time': 0.250909091, 'prompt_time': 0.001880436, 'queue_time': 0.101555339, 'total_time': 0.252789527}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_41c250edc7', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6e8815c-aa8e-4698-b997-b05bb3d98fd1-0', usage_metadata={'input_tokens': 40, 'output_tokens': 69, 'total_tokens': 109})

##### in final let's call LangSmith perform Traking

### Check if the Chatbot Remembers something

In [10]:
query = [
    HumanMessage(content='What is my favorite color ?')
    ]
chatbot.invoke(query)

AIMessage(content="I don't have any information about your personal preferences, including your favorite color. I'm a large language model, I don't have the ability to know or recall individual user preferences. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your favorite color with me, I'd be happy to chat with you about it!", response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 41, 'total_tokens': 125, 'completion_time': 0.305454545, 'prompt_time': 0.002365242, 'queue_time': 0.100352798, 'total_time': 0.307819787}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_8dd9fca28c', 'finish_reason': 'stop', 'logprobs': None}, id='run-71605484-f9d9-4d91-82b4-218b5682f765-0', usage_metadata={'input_tokens': 41, 'output_tokens': 84, 'total_tokens': 125})

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [12]:
chatBotMemory = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in chatBotMemory:
        chatBotMemory[session_id] = ChatMessageHistory()
    return chatBotMemory[session_id]
chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot,
    get_session_history
)

In [13]:
session1 = {"configurable" : {"session_id" : "001"}}

In [14]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is Green")],
    config = session1
)

responseFromChatbot.content

"Green is a wonderful color. It's a calming and natural color that can evoke feelings of balance and growth. It's also the color of nature, reminiscent of lush forests, rolling hills, and sparkling emeralds.\n\nWhat is it about the color green that you like the most? Is it the way it can symbolize harmony and freshness, or perhaps the way it can represent energy and vitality? Or maybe you just love the way it looks and the way it makes you feel?\n\nDo you have a favorite shade of green, such as lime, mint, or olive? Or do you like them all equally?"

In [15]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite Color ?")],
    config = session1
)

responseFromChatbot.content

'Your favorite color is Green. You told me that earlier.'

In [16]:
session2 = {"configurable" : {"session_id" : "002"}}

In [17]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite Color ?")],
    config = session2
)

responseFromChatbot.content

"I don't have any information about your personal preferences, including your favorite color. I'm a large language model, I don't have the ability to know or remember individual preferences. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. Would you like to tell me your favorite color?"